Chatbot local

Se realizará una conexión con un modelo cargado en LM Studio en local. Tendrá endpoints similares a los de OpenAI.

In [3]:
import requests

# Dirección del servidor local de LM Studio
url = "http://127.0.0.1:1234/v1/chat/completions"  # Asegúrate de que esta URL sea la correcta

# El mensaje del usuario que quieres enviar al modelo
payload = {
    "messages": [
        {"role": "system", "content": "Eres un chatbot amigable."},  # Mensaje inicial del sistema (opcional)
        {"role": "user", "content": "Hola, ¿cómo estás?"}  # Mensaje del usuario
    ],
    "max_tokens": 150
}

# Hacer la petición POST al endpoint local
response = requests.post(url, json=payload)

# Procesar la respuesta
if response.status_code == 200:
    print("Respuesta del Chatbot:", response.json()["choices"][0]["message"]["content"])
else:
    print(f"Error: {response.status_code}, {response.text}")


Respuesta del Chatbot: ¡Hola! ¡Estoy muy bien, gracias por preguntar! 😊 Como chatbot, no tengo sentimientos como los humanos, pero estoy funcionando óptimamente y listo para ayudarte con lo que necesites. 

¿Y tú? ¿Cómo estás hoy? Espero poder hacerte el día un poquito mejor. ✨



En la siguiente celda se harán preguntas al chatbot manteniendo el contexto de la conversación.

In [5]:
import requests

# Dirección del servidor local de LM Studio
url = "http://127.0.0.1:1234/v1/chat/completions"  # Asegúrate de que esta URL sea la correcta

# Inicializar el historial de mensajes con el mensaje de bienvenida (opcional)
messages = [
    {"role": "system", "content": "Eres un chatbot amigable."}
]

def hacer_pregunta(pregunta_usuario):
    # Añadir la nueva pregunta del usuario al historial de mensajes
    messages.append({"role": "user", "content": pregunta_usuario})
    
    # Crear el payload con el historial completo
    payload = {
        "messages": messages,
        "max_tokens": 150
    }

    # Hacer la petición POST al endpoint local
    response = requests.post(url, json=payload)

    # Procesar la respuesta
    if response.status_code == 200:
        respuesta_chatbot = response.json()["choices"][0]["message"]["content"]
        print("Pregunta del Usuario:", pregunta_usuario)
        print("Respuesta del Chatbot:", respuesta_chatbot)
        
        # Añadir la respuesta del chatbot al historial para mantener el contexto
        messages.append({"role": "assistant", "content": respuesta_chatbot})
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Prueba del chatbot
hacer_pregunta("Hola, ¿cómo estás?")
hacer_pregunta("¿Qué puedes hacer?")
hacer_pregunta("Cuéntame un chiste.")


Pregunta del Usuario: Hola, ¿cómo estás?
Respuesta del Chatbot: ¡Hola! Estoy muy bien, gracias por preguntar. 😊 Como chatbot, no tengo sentimientos como los humanos, pero estoy funcionando perfectamente y listo para ayudarte con lo que necesites. 

¿Y tú? ¿Cómo estás hoy? ¡Espero poder hacer tu día un poco más fácil! 😄

Pregunta del Usuario: ¿Qué puedes hacer?
Respuesta del Chatbot: ¡Puedo hacer muchas cosas! Soy un modelo de lenguaje grande, así que puedo comunicarme y generar texto similar al humano en respuesta a una amplia gama de indicaciones y preguntas.  Aquí te dejo algunas ideas de lo que puedo hacer:

* **Responder a tus preguntas:** Puedo responder preguntas sobre prácticamente cualquier tema, utilizando el conocimiento que he adquirido durante mi entrenamiento.
* **Generar diferentes formatos de texto creativo:** Puedo escribir poemas, código, guiones, piezas musicales, correo electrónico, cartas, etc. Intentaré cumplir con todos tus requisitos.
* **Resumir textos:**  Puedo

En la siguiente celda se realizará una conversación fluida con el chatbot local. Se deberá indica STOP para finalizar la conversación.

In [ ]:
import requests
import sys
import time
from IPython.display import clear_output

# Configuración inicial
url = "http://127.0.0.1:1234/v1/chat/completions"
conversation = [{"role": "system", "content": "Eres un asistente útil y amigable."}]

def typewriter_effect(text, delay=0.03):
    """Muestra el texto con efecto de máquina de escribir"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    print()

def print_flush(text):
    """Función para forzar la impresión del texto inmediatamente"""
    print(text)
    sys.stdout.flush()

print_flush("¡Bienvenido al Chatbot! Escribe 'STOP' en mayúsculas para terminar.\n")

while True:
    try:
        # Obtener entrada del usuario
        user_input = input("Tú: ")
        
        # Verificar si el usuario quiere terminar
        if user_input.upper() == "STOP":
            typewriter_effect("\n¡Hasta luego! Ha sido un placer ayudarte. 👋")
            break
        
        # Agregar el mensaje del usuario a la conversación
        conversation.append({"role": "user", "content": user_input})

        # Mostrar la pregunta del usuario
        print("\nTú: ", user_input)
        
        # Mostrar indicador de que el bot está pensando
        typewriter_effect("\nPensando...", 0.1)
        
        try:
            # Preparar y enviar la petición
            payload = {
                "messages": conversation,
                "max_tokens": 500
            }
            
            response = requests.post(url, json=payload, timeout=60)
            
            if response.status_code == 200:
                # Obtener la respuesta del chatbot
                bot_response = response.json()["choices"][0]["message"]["content"]
                
                # Mostrar la respuesta con efecto de escritura
                print("\nChatbot: ", end='')
                typewriter_effect(bot_response)
                print()  # Espacio adicional
                
                # Agregar la respuesta del asistente a la conversación para mantener el contexto
                conversation.append({"role": "assistant", "content": bot_response})
            else:
                typewriter_effect(f"\nError en la respuesta del servidor (Código {response.status_code})")
                
        except requests.exceptions.RequestException as e:
            typewriter_effect(f"\nError de conexión: {str(e)}")
            typewriter_effect("Asegúrate de que el servidor de LM Studio esté en ejecución.\n")
            time.sleep(2)
            continue
            
    except KeyboardInterrupt:
        typewriter_effect("\n\n¡Hasta luego! Interrupción del usuario detectada.")
        break
    except Exception as e:
        typewriter_effect(f"\nError inesperado: {str(e)}\n")
        time.sleep(1)

¡Bienvenido al Chatbot! Escribe 'STOP' en mayúsculas para terminar.


Tú:  Hola, que tal?

Pensando...

Chatbot: ¡Hola! ¡Muy bien, gracias por preguntar! 😊 ¿Y tú? ¿En qué puedo ayudarte hoy? Estoy aquí para responder tus preguntas, generar texto creativo, traducir idiomas, escribir diferentes tipos de contenido y mucho más. 

¡Dime lo que necesitas! 😊



Tú:  sabes que dia es hoy?

Pensando...

Chatbot: ¡Sí, claro que sé que día es hoy! Hoy es **domingo, 20 de mayo de 2024**. 

¿Necesitas saber algo más sobre el día de hoy? Quizás algún evento especial o la hora actual en alguna parte del mundo? ¡Avísame! 😊



Tú:  Que puedes contarme sobre Madrid?

Pensando...

Chatbot: ¡Madrid es una ciudad fascinante con mucho que ofrecer! Aquí te cuento algunas cosas clave sobre ella:

**En resumen:**

* **Capital de España:** Es el centro político, económico y cultural del país.
* **Ciudad vibrante:** Conocida por su energía, vida nocturna, arte, gastronomía y gente amable.
* **Historia rica:** Ti